In [1]:
from ultralytics import YOLO
import os
import random
import shutil
import numpy as np
import pandas as pd
import cv2
import yaml
import matplotlib.pyplot as plt
import glob
import sklearn
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
# annotation情報の確認
df = pd.read_csv("data/train_solution_bounding_boxes (1).csv")
df

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422
...,...,...,...,...,...
554,vid_4_9860.jpg,0.000000,198.321729,49.235251,236.223284
555,vid_4_9880.jpg,329.876184,156.482351,536.664239,250.497895
556,vid_4_9900.jpg,0.000000,168.295823,141.797524,239.176652
557,vid_4_9960.jpg,487.428988,172.233646,616.917699,228.839864


In [4]:
files = [file for file in df["image"].unique()]
files

['vid_4_1000.jpg',
 'vid_4_10000.jpg',
 'vid_4_10040.jpg',
 'vid_4_10020.jpg',
 'vid_4_10060.jpg',
 'vid_4_10100.jpg',
 'vid_4_10120.jpg',
 'vid_4_10140.jpg',
 'vid_4_1020.jpg',
 'vid_4_1040.jpg',
 'vid_4_10480.jpg',
 'vid_4_10500.jpg',
 'vid_4_10520.jpg',
 'vid_4_1060.jpg',
 'vid_4_10960.jpg',
 'vid_4_10980.jpg',
 'vid_4_11000.jpg',
 'vid_4_11020.jpg',
 'vid_4_11240.jpg',
 'vid_4_11260.jpg',
 'vid_4_11280.jpg',
 'vid_4_11380.jpg',
 'vid_4_11400.jpg',
 'vid_4_11420.jpg',
 'vid_4_11440.jpg',
 'vid_4_11900.jpg',
 'vid_4_11880.jpg',
 'vid_4_11920.jpg',
 'vid_4_11940.jpg',
 'vid_4_11960.jpg',
 'vid_4_11980.jpg',
 'vid_4_12000.jpg',
 'vid_4_12040.jpg',
 'vid_4_12100.jpg',
 'vid_4_12060.jpg',
 'vid_4_12080.jpg',
 'vid_4_12120.jpg',
 'vid_4_12140.jpg',
 'vid_4_12160.jpg',
 'vid_4_12180.jpg',
 'vid_4_12200.jpg',
 'vid_4_12220.jpg',
 'vid_4_12240.jpg',
 'vid_4_12260.jpg',
 'vid_4_12280.jpg',
 'vid_4_12300.jpg',
 'vid_4_12320.jpg',
 'vid_4_12340.jpg',
 'vid_4_12360.jpg',
 'vid_4_12380.jpg',
 'vi

In [16]:
# 画像ファイルのリスト(実際の.jpgファイル名から取得)
# フォルダパス
data = "data/training_images/"

# フォルダ内のファイルを取得
files = os.listdir(data)

# .jpgファイルのみを抽出
jpg_files = [f for f in files if f.endswith(".jpg")]

# ファイル名をリストに格納
filenames = [os.path.join(data, f) for f in jpg_files]

# 結果を出力
print(jpg_files)

['vid_4_1000.jpg', 'vid_4_10000.jpg', 'vid_4_10020.jpg', 'vid_4_10040.jpg', 'vid_4_10060.jpg', 'vid_4_10080.jpg', 'vid_4_10100.jpg', 'vid_4_10120.jpg', 'vid_4_10140.jpg', 'vid_4_10160.jpg', 'vid_4_10180.jpg', 'vid_4_1020.jpg', 'vid_4_10200.jpg', 'vid_4_10220.jpg', 'vid_4_10240.jpg', 'vid_4_10260.jpg', 'vid_4_10280.jpg', 'vid_4_10300.jpg', 'vid_4_10320.jpg', 'vid_4_10340.jpg', 'vid_4_10360.jpg', 'vid_4_10380.jpg', 'vid_4_1040.jpg', 'vid_4_10400.jpg', 'vid_4_10420.jpg', 'vid_4_10440.jpg', 'vid_4_10460.jpg', 'vid_4_10480.jpg', 'vid_4_10500.jpg', 'vid_4_10520.jpg', 'vid_4_10540.jpg', 'vid_4_10560.jpg', 'vid_4_10580.jpg', 'vid_4_1060.jpg', 'vid_4_10600.jpg', 'vid_4_10620.jpg', 'vid_4_10640.jpg', 'vid_4_10660.jpg', 'vid_4_10680.jpg', 'vid_4_10700.jpg', 'vid_4_10720.jpg', 'vid_4_10740.jpg', 'vid_4_10760.jpg', 'vid_4_10780.jpg', 'vid_4_1080.jpg', 'vid_4_10800.jpg', 'vid_4_10820.jpg', 'vid_4_10840.jpg', 'vid_4_10860.jpg', 'vid_4_10880.jpg', 'vid_4_10900.jpg', 'vid_4_10920.jpg', 'vid_4_10940.jpg

In [17]:
files_train, files_val = train_test_split(files, test_size=0.2, random_state=0, shuffle=True)
print(files_train)
print(len(files_train))
print(files_val)
print(len(files_val))

['vid_4_20780.jpg', 'vid_4_19240.jpg', 'vid_4_21600.jpg', 'vid_4_4620.jpg', 'vid_4_9080.jpg', 'vid_4_19480.jpg', 'vid_4_24920.jpg', 'vid_4_21980.jpg', 'vid_4_7120.jpg', 'vid_4_12380.jpg', 'vid_4_23500.jpg', 'vid_4_10660.jpg', 'vid_4_26180.jpg', 'vid_4_7620.jpg', 'vid_4_18200.jpg', 'vid_4_13640.jpg', 'vid_4_13480.jpg', 'vid_4_10720.jpg', 'vid_4_19420.jpg', 'vid_4_28220.jpg', 'vid_4_25920.jpg', 'vid_4_22140.jpg', 'vid_4_16520.jpg', 'vid_4_19400.jpg', 'vid_4_17100.jpg', 'vid_4_23360.jpg', 'vid_4_21860.jpg', 'vid_4_10320.jpg', 'vid_4_20880.jpg', 'vid_4_8720.jpg', 'vid_4_17380.jpg', 'vid_4_9600.jpg', 'vid_4_20120.jpg', 'vid_4_11140.jpg', 'vid_4_11480.jpg', 'vid_4_26580.jpg', 'vid_4_14380.jpg', 'vid_4_7480.jpg', 'vid_4_14140.jpg', 'vid_4_29760.jpg', 'vid_4_21420.jpg', 'vid_4_14520.jpg', 'vid_4_22600.jpg', 'vid_4_9820.jpg', 'vid_4_28340.jpg', 'vid_4_19380.jpg', 'vid_4_10020.jpg', 'vid_4_3860.jpg', 'vid_4_20040.jpg', 'vid_4_16220.jpg', 'vid_4_22820.jpg', 'vid_4_11820.jpg', 'vid_4_2000.jpg', 'v

## yoloのformatに修正

In [18]:
# formatを変更したデータの出力先
dir = "./dataset/"
images_train = dir + "train/images/"
labels_train = dir + "train/labels/"
images_val = dir + "valid/images/"
labels_val = dir + "valid/labels/"
# images_test = dir + "test/images/"
# labels_test = dir + "test/labels/"

In [19]:
# formatを変更したデータの出力先を作成
os.makedirs(images_train, exist_ok=True)
os.makedirs(labels_train, exist_ok=True)
os.makedirs(images_val, exist_ok=True)
os.makedirs(labels_val, exist_ok=True)
# os.makedirs(images_test, exist_ok=True)
# os.makedirs(labels_test, exist_ok=True)

In [21]:
# yolo形式のimageフォルダに.jpgをコピー
for file in glob.glob(data+"*.jpg"):
    # print(file)
    fname =os.path.basename(file)
    # print(fname)
    # 訓練画像
    if fname in files_train:
        shutil.copy(file, images_train)
    # 検証画像
    else:
        shutil.copy(file, images_val)

## labelフォルダを作成

In [24]:
def get_annotation_file(image_file, labels_train, labels_val):
  """
  画像ファイル名に応じて、アノテーションファイルのパスを返す関数

  Args:
      image_file (str): 画像ファイル名
      labels_train (str): トレーニングデータのアノテーションファイル出力先のフォルダパス
      labels_val (str): バリデーションデータのアノテーションファイル出力先のフォルダパス

  Returns:
      str: アノテーションファイルのパス
  """
  if image_file in files_train:
    annotation_file = os.path.join(labels_train, image_file.replace('.jpg', '.txt'))
  else:
    annotation_file = os.path.join(labels_val, image_file.replace('.jpg', '.txt'))
  return annotation_file

In [27]:
h = 380
w = 676

for _, row in df.iterrows():
  image_file = row["image"]
  # print(image_file)
  class_id = 0
  x = row["xmin"]
  y = row["ymin"]
  width = row["xmax"] - row["xmin"]
  height = row["ymax"] - row["ymin"]
  x_center = (x + (width / 2)) / w  # 中央座標の計算
  y_center = (y + (height / 2)) / h  # 中央座標の計算
  width /= w  # 正規化
  height /= h  # 正規化

  # ファイルパスを判別する論理を関数へ分割 (修正箇所)
  annotation_file = get_annotation_file(image_file, labels_train, labels_val)

  with open(annotation_file, 'a') as ann_file:
    ann_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

vid_4_1000.jpg
vid_4_10000.jpg
vid_4_10040.jpg
vid_4_10020.jpg
vid_4_10060.jpg
vid_4_10100.jpg
vid_4_10120.jpg
vid_4_10140.jpg
vid_4_1020.jpg
vid_4_1040.jpg
vid_4_10480.jpg
vid_4_10500.jpg
vid_4_10520.jpg
vid_4_1060.jpg
vid_4_10960.jpg
vid_4_10980.jpg
vid_4_11000.jpg
vid_4_11020.jpg
vid_4_11240.jpg
vid_4_11260.jpg
vid_4_11280.jpg
vid_4_11380.jpg
vid_4_11400.jpg
vid_4_11420.jpg
vid_4_11440.jpg
vid_4_11900.jpg
vid_4_11880.jpg
vid_4_11920.jpg
vid_4_11940.jpg
vid_4_11960.jpg
vid_4_11980.jpg
vid_4_12000.jpg
vid_4_12040.jpg
vid_4_12100.jpg
vid_4_12060.jpg
vid_4_12080.jpg
vid_4_12120.jpg
vid_4_12140.jpg
vid_4_12140.jpg
vid_4_12160.jpg
vid_4_12160.jpg
vid_4_12160.jpg
vid_4_12180.jpg
vid_4_12180.jpg
vid_4_12180.jpg
vid_4_12200.jpg
vid_4_12200.jpg
vid_4_12220.jpg
vid_4_12240.jpg
vid_4_12240.jpg
vid_4_12260.jpg
vid_4_12260.jpg
vid_4_12280.jpg
vid_4_12280.jpg
vid_4_12300.jpg
vid_4_12300.jpg
vid_4_12300.jpg
vid_4_12300.jpg
vid_4_12320.jpg
vid_4_12320.jpg
vid_4_12320.jpg
vid_4_12340.jpg
vid_4_12340.